# IMPORTAÇÕES

In [ ]:
import pandas as pd
import numpy as np

from copy import copy, deepcopy
from typing import List

# FUNÇÕES

In [ ]:
class Environment:
  def __init__(self, root, rel_path):
    self.root = root            # Pasta raiz
    self.rel_path = rel_path    # Caminho relativo

  def get_file_path(self):
    """
    Retorna caminho do arquivo.
    :return: Caminho do arquivo.
    """
    parts = [self.root]
    parts.extend(self.rel_path)
    return "/".join(parts)

  def clone(self, mode="shallow"):
    """
    Retorna uma cópia do objeto. O padrão é shallow copy.
    :param mode: Modo de cópia ("shallow" ou "deep").
    :return: Cópia do objeto.
    """
    if mode == "deep":
      return deepcopy(self)
    return copy(self)

def df_head(df: pd.DataFrame, num: int = 5) -> None:
  """
  Exibe as primeiras linhas do DataFrame.

  :param df: DataFrame a ser exibido.
  :param num: Número de linhas a serem exibidas.
  """
  display(df.head(num))
  print(f"Shape: {df.shape}")

def df_show_domain(df: pd.DataFrame, columns: List[str]) -> None:
  """
  Exibe o tipo e os valores possíveis de columns segundo a ordem da lista de
  colunas.

  :param df: DataFrame a ter os domínios exibidos.
  :param columns: Lista dos nomes das colunas a serem exibidas.
  :return: None
  """
  for col in columns:
    print(f"Coluna:   {col}")
    print(f"dtype:    {df[col].dtype}")
    print(f"Domínio:  {df[col].unique()}\n")

def df_show_null(df: pd.DataFrame) -> None:
  """
  Exibe o número de valores nulos por coluna.

  :param df: DataFrame a ter os domínios exibidos.
  :return: None
  """
  print("# Número de valores nulos por coluna:")
  display(df.isnull().sum())

# SETUP DO AMBIENTE

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# FORMATO DO DATASET

Foi necessário reformatar o arquivo CSV original, pois seu formato mostrou-se imcompreensível pelo `pandas`. Diante disso, fez-se uso gratuito do chat GPT 4o-mini com o seguinte _prompt_:


> _Poderia formatar adequadamente o suposto csv que lhe enviarei? Quando fui abri-lo no pandas, os dados não foram processados adequadamente._


Depois de enviá-lo ao modelo, obteve-se a resposta abaixo ("Diagnóstico do Modelo"), além de um arquivo que se contatou devidamente formatado depois de rigorosa inspeção.

## Diagnóstico do Modelo

O arquivo CSV que você enviou está com formatação não padrão. Aqui está o diagnóstico:

- A **primeira linha** contém os anos como cabeçalhos, separados por vírgulas.
- Porém, há um bloco de **vários pontos e vírgulas (`;`)** após os anos, provavelmente tentando simular colunas vazias.
- As linhas seguintes contêm os estados e suas respectivas séries históricas de emissões, mas parecem desalinhadas por conta dessa estrutura híbrida de vírgulas e ponto e vírgula.

Vamos fazer a correção para deixar o CSV legível pelo `pandas` com as colunas corretas. Vou limpar os `;` extras e reconstruir um DataFrame bem formatado.

---

O arquivo continha vários problemas de formatação:

- A **primeira linha estava entre aspas triplas (`"""`)** e com muitos `;` no final.
- Os dados de cada linha estavam todos lidos como uma única string, sem separação adequada entre os campos.
- Isso fez com que o `pandas` agrupasse os dados incorretamente e os títulos de colunas ficassem mal formatados.

Vou reestruturar o CSV para deixá-lo assim:

- Coluna 1: `Estado`
- Colunas seguintes: anos de **1990 a 2023**
- Valores: emissões de carbono (como `float`)

Em seguida, disponibilizarei o CSV corrigido. Vamos ajustar e salvar.

---

O arquivo foi corrigido com sucesso! Agora ele possui:

- A coluna **"Estado"** com os nomes limpos.
- Uma coluna para cada ano de **1990 a 2023**, com os valores numéricos de emissões.
- Todos os ruídos de aspas e ponto-e-vírgula foram removidos.


# OBTENÇÃO DO DATASET

In [ ]:
ROOT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets"
env_in = Environment(ROOT, ["raw", "SEEG", "SEEG_limpo_gpt.csv"])

# Obtém DataFrame
df_raw = pd.read_csv(env_in.get_file_path())

# Exibe resultado
df_head(df_raw)
df_show_domain(df_raw, df_raw.columns)

,Estado,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Pará,3.713604e+08,3.456612e+08,4.887939e+08,3.903480e+08,4.096926e+08,5.709845e+08,4.608000e+08,5.624068e+08,4.809973e+08,...,2.608387e+08,2.970738e+08,3.589159e+08,2.965203e+08,3.345482e+08,4.228341e+08,4.108781e+08,5.389185e+08,4.390504e+08,3.122605e+08
1,Mato Grosso,2.991454e+08,3.080704e+08,4.123194e+08,3.959249e+08,5.462023e+08,6.462798e+08,5.358809e+08,5.237706e+08,5.331691e+08,...,2.259236e+08,2.484652e+08,2.127614e+08,2.206877e+08,2.319696e+08,2.789041e+08,2.436733e+08,2.646122e+08,3.368483e+08,2.985711e+08
2,Maranhão,1.551959e+08,1.437850e+08,2.037116e+08,1.642882e+08,1.526168e+08,1.432679e+08,1.394976e+08,1.288871e+08,1.897565e+08,...,9.831621e+07,1.063022e+08,1.066347e+08,9.558932e+07,9.302000e+07,1.056525e+08,1.193823e+08,1.171970e+08,1.538411e+08,1.734238e+08
3,Minas Gerais,1.718919e+08,1.803260e+08,1.676099e+08,1.711714e+08,1.590313e+08,1.647119e+08,1.593408e+08,1.634456e+08,1.568034e+08,...,1.758563e+08,1.727664e+08,1.643221e+08,1.569789e+08,1.546153e+08,1.557282e+08,1.569076e+08,1.665208e+08,1.651998e+08,1.697552e+08
4,São Paulo,1.188844e+08,1.254230e+08,1.220580e+08,1.254794e+08,1.377703e+08,1.401017e+08,1.388923e+08,1.400873e+08,1.409485e+08,...,1.666131e+08,1.584306e+08,1.498436e+08,1.535929e+08,1.478240e+08,1.473847e+08,1.424277e+08,1.498882e+08,1.524313e+08,1.547762e+08


Shape: (67, 35)
Coluna:   Estado
dtype:    object
Domínio:  ['Pará' 'Mato Grosso' 'Maranhão' 'Minas Gerais' 'São Paulo' 'Bahia'
 'Rondônia' 'Amazonas' 'Goiás' 'Tocantins' 'Rio Grande do Sul'
 'Mato Grosso do Sul' 'Paraná' 'Não Alocado' 'Rio de Janeiro'
 'Santa Catarina' 'Piauí' 'Ceará' 'Acre' 'Espírito Santo' 'Roraima'
 'Pernambuco' 'Paraíba' 'Rio Grande do Norte' 'Alagoas' 'Sergipe'
 'Distrito Federal' 'Amapá' ';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'
 ';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;3.122.605.289.059.790']

Coluna:   1990
dtype:    float64
Domínio:  [3.71360389e+08 2.99145398e+08 1.55195860e+08 1.71891875e+08
 1.18884393e+08 1.09641990e+08 7.09765543e+07 2.16067090e+07
 1.06924460e+08 6.93007954e+07 1.01755728e+08 1.50843672e+08
 7.96538411e+07 6.90703712e+07 3.13486549e+07 4.10912207e+07
 1.64931972e+07 2.25369725e+07 2.13089613e+07 2.11960112e+07
 9.23311071e+06 1.76952204e+07 8.94846559e+06 7.63836858e+06
 7.99389647e+06 6.66255374e+06 3.03502159e+06 3.45055538e+06
            nan

# CONFORMAÇÃO DE DOMÍNIO

In [ ]:
# Dicionário de sigla dos estados (inclui o Distrito Federal)
DICT_STATES = {
    "Acre": "AC",
    "Alagoas": "AL",
    "Amapá": "AP",
    "Amazonas": "AM",
    "Bahia": "BA",
    "Ceará": "CE",
    "Distrito Federal": "DF",
    "Espírito Santo": "ES",
    "Goiás": "GO",
    "Maranhão": "MA",
    "Mato Grosso": "MT",
    "Mato Grosso do Sul": "MS",
    "Minas Gerais": "MG",
    "Pará": "PA",
    "Paraíba": "PB",
    "Paraná": "PR",
    "Pernambuco": "PE",
    "Piauí": "PI",
    "Rio de Janeiro": "RJ",
    "Rio Grande do Norte": "RN",
    "Rio Grande do Sul": "RS",
    "Rondônia": "RO",
    "Roraima": "RR",
    "Santa Catarina": "SC",
    "São Paulo": "SP",
    "Sergipe": "SE",
    "Tocantins": "TO"
}
STATE_COL = "Estado" # Nome da coluna respectiva a estados

# Mapeia nome extenso em sigla dos estados
df_filtered = df_raw.copy()
# Se não houver correspondência, valor em STATE_COL recebe pd.NA (Not Available)
df_filtered[STATE_COL] = df_raw[STATE_COL].apply(lambda x: DICT_STATES.get(x, pd.NA))

# Remove colunas sem estado correspondente
df_filtered = df_filtered.dropna(subset=[STATE_COL])

# Reordena e reindexa
df = df_filtered.sort_values(by=[STATE_COL]).reset_index(drop=True)

display(df_filtered)

,Estado,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,PA,3.713604e+08,3.456612e+08,4.887939e+08,3.903480e+08,4.096926e+08,5.709845e+08,4.608000e+08,5.624068e+08,4.809973e+08,...,2.608387e+08,2.970738e+08,3.589159e+08,2.965203e+08,3.345482e+08,4.228341e+08,4.108781e+08,5.389185e+08,4.390504e+08,3.122605e+08
1,MT,2.991454e+08,3.080704e+08,4.123194e+08,3.959249e+08,5.462023e+08,6.462798e+08,5.358809e+08,5.237706e+08,5.331691e+08,...,2.259236e+08,2.484652e+08,2.127614e+08,2.206877e+08,2.319696e+08,2.789041e+08,2.436733e+08,2.646122e+08,3.368483e+08,2.985711e+08
2,MA,1.551959e+08,1.437850e+08,2.037116e+08,1.642882e+08,1.526168e+08,1.432679e+08,1.394976e+08,1.288871e+08,1.897565e+08,...,9.831621e+07,1.063022e+08,1.066347e+08,9.558932e+07,9.302000e+07,1.056525e+08,1.193823e+08,1.171970e+08,1.538411e+08,1.734238e+08
3,MG,1.718919e+08,1.803260e+08,1.676099e+08,1.711714e+08,1.590313e+08,1.647119e+08,1.593408e+08,1.634456e+08,1.568034e+08,...,1.758563e+08,1.727664e+08,1.643221e+08,1.569789e+08,1.546153e+08,1.557282e+08,1.569076e+08,1.665208e+08,1.651998e+08,1.697552e+08
4,SP,1.188844e+08,1.254230e+08,1.220580e+08,1.254794e+08,1.377703e+08,1.401017e+08,1.388923e+08,1.400873e+08,1.409485e+08,...,1.666131e+08,1.584306e+08,1.498436e+08,1.535929e+08,1.478240e+08,1.473847e+08,1.424277e+08,1.498882e+08,1.524313e+08,1.547762e+08
5,BA,1.096420e+08,1.129881e+08,1.119794e+08,1.019283e+08,1.001258e+08,1.038018e+08,9.602919e+07,8.918156e+07,8.698805e+07,...,9.249481e+07,8.543389e+07,8.431134e+07,7.866862e+07,8.697221e+07,9.265844e+07,1.003024e+08,1.120670e+08,1.202893e+08,1.349629e+08
6,RO,7.097655e+07,1.160730e+08,1.454002e+08,1.458490e+08,1.859054e+08,2.394983e+08,2.229381e+08,1.598575e+08,1.481754e+08,...,9.115823e+07,1.180436e+08,1.319840e+08,1.251358e+08,1.148093e+08,1.490945e+08,1.292419e+08,1.524549e+08,1.549645e+08,9.963256e+07
7,AM,2.160671e+07,2.318207e+07,3.393175e+07,3.312819e+07,3.480961e+07,3.760181e+07,4.373628e+07,6.041158e+07,7.492997e+07,...,5.555070e+07,7.872848e+07,1.094925e+08,8.003458e+07,7.560043e+07,1.268535e+08,1.172963e+08,1.863024e+08,2.053111e+08,9.292016e+07
8,GO,1.069245e+08,1.046961e+08,1.124687e+08,1.056109e+08,1.032288e+08,1.059306e+08,9.108592e+07,8.881631e+07,8.569714e+07,...,8.723152e+07,8.742488e+07,8.362872e+07,8.438724e+07,8.365465e+07,8.738269e+07,9.072666e+07,9.001982e+07,9.176990e+07,9.283197e+07
9,TO,6.930080e+07,6.748771e+07,8.608896e+07,7.778089e+07,6.366640e+07,9.041227e+07,7.148214e+07,6.384949e+07,5.773779e+07,...,5.911618e+07,6.356787e+07,5.664445e+07,5.343780e+07,5.470605e+07,5.841257e+07,5.666977e+07,5.296716e+07,6.500293e+07,8.870288e+07


# VALORES FALTANTES

Como não há valores faltantes, não há que se realizar qualquer imputação ou remoção.

In [ ]:
df_show_null(df_filtered)

# Número de valores nulos por coluna:


,0
Estado,0
1990,0
1991,0
1992,0
1993,0
1994,0
1995,0
1996,0
1997,0
1998,0


# _OUTLIERS_

Quanto a _outliers_, optou-se por deliberadamente não os remover, uma vez que a emissão de dióxido de carbono na atmosfera é sensível à atividade antrópica, de natureza volátil e imprevisível, porque atrelada a dezenas de variáveis: disposições legais do País, demanda por madeira, governo vigente etc.

# REESTRUTURAÇÃO

In [ ]:
# Transforma colunas de ano em atributos
df_melted = df_filtered.melt(
    id_vars=[STATE_COL],
    var_name="_ano",
    value_name="car_c02_emitido"
)
# Renomeia colunas
df_melted.rename(columns={STATE_COL: "_estado"}, inplace=True)

display(df_melted)

,_estado,_ano,car_c02_emitido
0,PA,1990,3.713604e+08
1,MT,1990,2.991454e+08
2,MA,1990,1.551959e+08
3,MG,1990,1.718919e+08
4,SP,1990,1.188844e+08
...,...,...,...
913,RN,2023,1.321502e+07
914,AL,2023,1.050894e+07
915,SE,2023,9.628775e+06
916,DF,2023,7.274557e+06


# EXTRAPOLAÇÃO

Como a granularidade temporal dos dados a alimentarem o modelo deveria ser mensal, fez-se, sob sugestão da professora Paula Maçaira, a seguinte extrapolação: não tendo sido possível coletar informações mês a mês sobre a emissão de dióxido de carbono, mas apenas ano a ano; decidiu-se replicar o valor anual ao longo dos meses, a fim de que refletissem o comportamento geral — algo semelhante à imputação da média, mas multiplicada pelo número de meses. Dessa forma, espera-se, o modelo ainda será capaz de identificar padrões, se bem que com menor teor de detalhe. Seja como for, ainda se procuram as informações com a devida granularidade, que poderão ser incorporadas ao projeto em futuras _sprints_.

In [ ]:
# PARÂMETROS AUXILIARES
MONTH_COUNT = 12                                   # Número de meses
INIT_VALUE = 1                                     # Índice inicial
END_VALUE = INIT_VALUE + MONTH_COUNT               # Índice final
MONTHS = [x for x in range(INIT_VALUE, END_VALUE)] # Vetor de índices
MONTH_COL = "_mes"                                 # Nome da coluna de meses

dfs = list()
df_aux = None

for month in MONTHS:
  df_aux = df_melted.copy()
  df_aux[MONTH_COL] = month # Adiciona coluna com o respectivo mês
  dfs.append(df_aux)        # Adiciona à lista de DataFrames

# Concatena a lista
df = pd.concat(dfs).reset_index(drop=True)

df_head(df)

,_estado,_ano,car_c02_emitido,_mes
0,PA,1990,3.713604e+08,1
1,MT,1990,2.991454e+08,1
2,MA,1990,1.551959e+08,1
3,MG,1990,1.718919e+08,1
4,SP,1990,1.188844e+08,1


Shape: (11016, 4)


# CORREÇÃO DE TIPOS

In [ ]:
# CORREÇÃO DE TIPOS
df["_estado"] = df["_estado"].astype("string")                  # Converte para string
df["_mes"] = df["_mes"].astype("Int32")                         # Converte para Int32 (nulificável)
df["_ano"] = df["_ano"].astype("Int32")                         # Converte para Int32 (nulificável)
df["car_c02_emitido"] = df["car_c02_emitido"].astype("Float64") # Converte para Float64 (nulificável)

# Exibe resultado
df_show_domain(df, df.columns)

Coluna:   _estado
dtype:    string
Domínio:  <StringArray>
['PA', 'MT', 'MA', 'MG', 'SP', 'BA', 'RO', 'AM', 'GO', 'TO', 'RS', 'MS', 'PR',
 'RJ', 'SC', 'PI', 'CE', 'AC', 'ES', 'RR', 'PE', 'PB', 'RN', 'AL', 'SE', 'DF',
 'AP']
Length: 27, dtype: string

Coluna:   _ano
dtype:    Int32
Domínio:  <IntegerArray>
[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Length: 34, dtype: Int32

Coluna:   car_c02_emitido
dtype:    Float64
Domínio:  <FloatingArray>
[371360388.81906617,  299145398.1764236, 155195859.97243583,
  171891874.7191574, 118884392.75664908, 109641990.14207718,
  70976554.29632598, 21606709.009402085,  106924459.7480254,
  69300795.38282482,
 ...
  29615770.90129041, 29566932.231589705, 28627986.022511743,
 26611322.086775333,  15476573.95553324, 13215018.982496202,
 10508940.506028544,  9628774.674436588,  7274556.750461329

# REORDENAÇÃO

In [ ]:
# Ordena de forma mais conveniente
df = df.sort_values(by=["_estado", "_ano", "_mes"]).reset_index(drop=True)
df = df[sorted(df.columns)]

# Exibe resultado
df_head(df, MONTH_COUNT)

,_ano,_estado,_mes,car_c02_emitido
0,1990,AC,1,21308961.330824
1,1990,AC,2,21308961.330824
2,1990,AC,3,21308961.330824
3,1990,AC,4,21308961.330824
4,1990,AC,5,21308961.330824
5,1990,AC,6,21308961.330824
6,1990,AC,7,21308961.330824
7,1990,AC,8,21308961.330824
8,1990,AC,9,21308961.330824
9,1990,AC,10,21308961.330824


Shape: (11016, 4)


# SALVAMENTO

In [ ]:
# SALVAMENTO
env_out = Environment(ROOT, ["cooked", "SEEG", "emissoes_c02_cooked.csv"])
df.to_csv(env_out.get_file_path(), index=False, encoding="utf-8")